In [1]:
# stanza.download("fr")
# stanza.download("nl")

In [2]:
from collections import Counter
import pandas as pd
import stanza
import re

In [29]:
en_gt = pd.read_csv('../ACTER/en/htfl/annotations/htfl_en_terms.ann', sep='	', engine='python',header=None)
en_preds = pd.read_csv('../results/en/ann/5_lem_roberta_ann_predictions.csv')
preds = list(en_preds['predictions'])
gts = list(en_gt[0])

In [30]:
def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

In [31]:
preds_len = [len(x.split(' ')) for x in preds]
gts_len = [len(x.split(' ')) for x in gts]
print(Counter(preds_len))
print(Counter(gts_len))

Counter({1: 734, 2: 46, 3: 2})
Counter({1: 1064, 2: 766, 3: 357, 4: 120, 5: 31, 6: 18, 7: 4, 8: 1})


In [32]:
max(gts, key=len)

'heart failure with preserved left ventricular ejection fraction'

In [33]:
# gts[gts.str.split(' ').len() == 3]

In [34]:
# P = TP/(TP+FP)
# R =  TP/(TP+FN)
def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    print(TP, FP, FN)
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score

precision, recall, f1_score = evaluation_metrics(preds, gts)
print(len(set(preds)))
print(len(set(gts)))
precision, recall, f1_score

372 410 1989
782
2361


(47.57, 15.76, 23.68)

Stanza

In [35]:
nlp = stanza.Pipeline(lang='en')# processors='tokenize,mwt,pos,lemma'

2021-04-15 12:58:55 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-04-15 12:58:55 INFO: Use device: cpu
2021-04-15 12:58:55 INFO: Loading: tokenize
2021-04-15 12:58:55 INFO: Loading: pos
2021-04-15 12:58:55 INFO: Loading: lemma
2021-04-15 12:58:55 INFO: Loading: depparse
2021-04-15 12:58:56 INFO: Loading: sentiment
2021-04-15 12:58:56 INFO: Loading: ner
2021-04-15 12:58:57 INFO: Done loading processors!


In [41]:
def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

In [42]:
preds_lem = lemma(preds)
gts_lem = lemma(gts)

print('After lemmatization')
print(len(set(preds_lem)))
print(len(set((gts_lem))))
print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

After lemmatization
743
2149
350 393 1799
(47.11, 16.29, 24.21)


In [43]:
set(preds_lem)

{"'s",
 'ablation',
 'absolute',
 'absorptiometry',
 'absorption',
 'accountable',
 'acetyl',
 'activate',
 'active',
 'activity',
 'adenosine',
 'adenoviral',
 'adenovirus',
 'adipokine',
 'administrative',
 'admission',
 'adrenergic',
 'adrenoceptor',
 'aerobic',
 'aetiology',
 'af',
 'after',
 'aging',
 'aim',
 'air pollution',
 'aldosterone',
 'aldosterone system',
 'ambulatory',
 'ami',
 'amino',
 'amyloidosis',
 'amyloidotic',
 'anaerobic',
 'anaerobic threshold',
 'anaesthesia',
 'anastomosis',
 'and',
 'anemia',
 'angina',
 'angiogenesis',
 'angiography',
 'angioplasty',
 'angiotensin',
 'ankle',
 'annular',
 'annular plane',
 'annular velocity',
 'annuloplasty',
 'annulus velocity',
 'anterior',
 'anti',
 'antiadrenergic',
 'antiarrhythmic',
 'antibody',
 'antiplatelet',
 'antirheumatic',
 'aortic',
 'aortic constriction',
 'aortic valve',
 'apical',
 'apoptosis',
 'appendicular',
 'arm',
 'arrest',
 'arrhythmia',
 'arterial',
 'arterial tonometry',
 'artery',
 'artery stenosi

In [44]:
set(gts_lem)

{'candesartan',
 'end stage hf',
 'neprilysin inhibitor',
 'wide qrs complex',
 'maximal ventricular power index',
 'idiopathic dilated cardiomyopathy',
 'ventricular function',
 'cardioverter defibrillator',
 'palliative care',
 'eplerenone',
 'irs2',
 'pressure ulcer',
 'multivariable analysis',
 'bat',
 'continuous wave doppler echocardiography',
 'mena expression',
 'energy metabolism',
 'hfref',
 'pro2',
 'percutaneous',
 'cs',
 'bhlh',
 'angiotensin ii',
 'anemic',
 'red blood cell',
 'edge to edge anastomosis',
 'infection',
 'aldosterone antagonist',
 'cardiac troponin t',
 'nitroxyl',
 'creatinine',
 'tricuspid valve',
 'tricuspid annular plane',
 "'s 12884",
 'plasma membrane translocation',
 '( 123)i meta iodobenzylguanidine',
 'raas',
 'frail',
 'pioglitazone',
 'gly4',
 'ras blocker',
 'allopurinol',
 'etidronate',
 'myosin',
 'ss',
 'sympathetic activity',
 'bottleneck stenting',
 'infarct',
 'stenosis',
 'secondary endpoint',
 'renin angiotensin aldosteron',
 'cha₂ds₂vas

In [45]:
# count_freq(set(preds_lem), set(gts_lem))

Counter({1: 692, 2: 48, 3: 3})
Counter({1: 857, 2: 672, 3: 379, 4: 148, 5: 46, 6: 30, 7: 13, 8: 3, 10: 1})


NLTK

In [ ]:
# from nltk.stem import PorterStemmer, WordNetLemmatizer
# port = PorterStemmer()
# wnl = WordNetLemmatizer()

# preds_stem = [port.stem(i) for i in preds]
# gts_stem = [port.stem(i) for i in gts]

# print('After stemming')
# print(len(set(preds_stem)))
# print(len(set(gts_stem)))
# print(evaluation_metrics(list(set(preds_stem)), list(set(gts_stem))))

# preds_lem = [wnl.lemmatize(i) for i in preds]
# gts_lem = [wnl.lemmatize(i) for i in gts]

# print('After lemmatization')
# print(len(set(preds_lem)))
# print(len(set((gts_lem))))
# print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

# preds_com = [port.stem(wnl.lemmatize(i)) for i in preds]
# gts_com = [port.stem(wnl.lemmatize(i)) for i in gts]

# print('After combined')
# print(len(set(preds_com)))
# print(len(set((gts_com))))
# evaluation_metrics(list(set(preds_com)), list(set(gts_com)))

# preds_ =[x for x in preds if len(x) >= 2]
# gts_ =[x for x in gts if len(x) >= 2]
# print(len(set(preds_)))
# print(len(gts))
# evaluation_metrics(set(preds_), gts)